# Assignment 4

In [102]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [103]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [110]:
def graph_identification():
    algorithms = []
    
    avg_cls = [nx.average_clustering(i) for i in P1_Graphs]
    avg_spl = [nx.average_shortest_path_length(i) for i in P1_Graphs]
    
    for i in range(len(P1_Graphs)):
        degrees = dict(P1_Graphs[i].degree())  # Convert DegreeView to a dictionary
        degree_values = sorted(set(degrees.values()))  # Extract values from the dictionary and convert to a list
        
        # print(f"Graph {i}, degree_values: {degree_values}")
        
        histogram = [list(degrees.values()).count(deg) / float(nx.number_of_nodes(P1_Graphs[i])) for deg in degree_values]
    
        # print(f"Graph {i}, histogram: {histogram}")
        
        if len(histogram) > 10:
            algorithms.append('PA')
        elif avg_cls[i] <= 0.4:
            algorithms.append('SW_H')
        else:
            algorithms.append('SW_L')
            
    return algorithms

# Assuming P1_Graphs is a list of graphs
graph_identification()


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

In [105]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [24]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [25]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [72]:
def salary_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    
    # Create a DataFrame from node attributes
    df = pd.DataFrame(list(G.nodes(data=True)), columns=['node_id', 'attributes'])
    
    # Extract features from attributes
    df['Department'] = df['attributes'].apply(lambda x: x['Department'])
    df['ClustCoef'] = pd.Series(nx.clustering(G))
    df['DegreeCent'] = pd.Series(nx.degree_centrality(G))
    df['Degree'] = pd.Series(nx.degree(G)).apply(lambda x: x[1]).astype(float)
    df['ClosenessCent'] = pd.Series([v for i,v in nx.closeness_centrality(G).items()])
    df['BetwCent'] = pd.Series(nx.betweenness_centrality(G))
    df['y'] = df['attributes'].apply(lambda x: x.get('ManagementSalary'))

    # Features for training
    df_train = df[~pd.isnull(df['y'])]
    X_train = df_train[['Department', 'ClustCoef', 'DegreeCent', 'Degree', 'ClosenessCent', 'BetwCent']]
    y_train = df_train['y']
    #print(X_train.dtypes)
    # Features for prediction
    df_pred = df[pd.isnull(df['y'])]
    X_pred = df_pred[['Department', 'ClustCoef', 'DegreeCent', 'Degree', 'ClosenessCent', 'BetwCent']]

    # Train a Gradient Boosting Classifier
    clf = GradientBoostingClassifier(n_estimators=600, learning_rate=0.1, max_depth=5, random_state=0)
    clf.fit(X_train, y_train)

    # Predict probabilities for nodes with missing 'ManagementSalary'
    probabilities = clf.predict_proba(X_pred)[:, 1]

    # Create a Pandas series with the probabilities and node IDs
    result_series = pd.Series(probabilities, index=df_pred['node_id'])

    return result_series

salary_predictions()

node_id
1      2.731656e-06
65     5.310321e-05
18     5.920893e-02
215    2.343217e-06
283    9.999565e-01
           ...     
691    5.492444e-08
788    2.454234e-07
944    5.492476e-08
798    5.995107e-08
808    3.769857e-08
Length: 252, dtype: float64

In [73]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [74]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [100]:
'''def new_connections_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    import pandas as pd

    # Step 1: Create a matrix of features for the edges in future_connections using NetworkX
    future_connections['PreferentialAttachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['CommonNeighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    future_connections['JaccardCoefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['ResourceAllocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]

    # Step 2: Train a classifier on the edges in future_connections that have 'Future Connection' data
    train_data = future_connections[~pd.isnull(future_connections['Future Connection'])]
    X_train = train_data[['PreferentialAttachment', 'CommonNeighbors', 'JaccardCoefficient', 'ResourceAllocation']]
    y_train = train_data['Future Connection']

    clf = GradientBoostingClassifier(n_estimators=600, learning_rate=0.1, max_depth=5, random_state=0)
    clf.fit(X_train, y_train)

    # Step 3: Predict the probability of future connection for edges with missing 'Future Connection' data
    test_data = future_connections[pd.isnull(future_connections['Future Connection'])]
    X_test = test_data[['PreferentialAttachment', 'CommonNeighbors', 'JaccardCoefficient', 'ResourceAllocation']]
    probabilities = clf.predict_proba(X_test)[:, 1]

    # Step 4: Return a series with the predicted probabilities
    result_series = pd.Series(probabilities, index=test_data.index)

    # Print the result_series
    return(result_series)
new_connections_predictions()'''

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import networkx as nx

# Assuming you have your future_connections and G defined

def new_connections_predictions():
    # Step 1: Create a matrix of features for the edges in future_connections using NetworkX
    future_connections['PreferentialAttachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['CommonNeighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    future_connections['JaccardCoefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['ResourceAllocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]

    # Step 2: Train a classifier on the edges in future_connections that have 'Future Connection' data
    train_data = future_connections[~pd.isnull(future_connections['Future Connection'])]
    X_train = train_data[['PreferentialAttachment', 'CommonNeighbors', 'JaccardCoefficient', 'ResourceAllocation']]
    y_train = train_data['Future Connection']

    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
    clf.fit(X_train, y_train)

    # Step 3: Predict the probability of future connection for edges with missing 'Future Connection' data
    test_data = future_connections[pd.isnull(future_connections['Future Connection'])]
    X_test = test_data[['PreferentialAttachment', 'CommonNeighbors', 'JaccardCoefficient', 'ResourceAllocation']]
    probabilities = clf.predict_proba(X_test)[:, 1]

    # Step 4: Return a series with the predicted probabilities
    result_series = pd.Series(probabilities, index=test_data.index)

    # Step 5: Calculate ROC-AUC score
    roc_auc = roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])

    # Print the ROC-AUC score
    #print(f"ROC-AUC Score: {roc_auc}")

    # Print the result_series
    return result_series

# Assuming future_connections and G are defined
new_connections_predictions()

(107, 348)    0.029567
(542, 751)    0.013001
(20, 426)     0.562763
(50, 989)     0.013001
(942, 986)    0.013001
                ...   
(165, 923)    0.013173
(673, 755)    0.013001
(939, 940)    0.013001
(555, 905)    0.013001
(75, 101)     0.016199
Length: 122112, dtype: float64

In [101]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"
